In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import re

import matplotlib.pyplot as plt 
import seaborn as sns
from wordcloud import WordCloud
import pyLDAvis
sns.set()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim


# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import stopwords

E:\Anaconda\envs\myenvnew\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
E:\Anaconda\envs\myenvnew\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [5]:
df = pd.read_csv('C:/Users/pande/Capstone/Profiles/data/df_processed_bio.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057 entries, 0 to 1056
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bio                 1057 non-null   object
 1   word_tokenized      1057 non-null   object
 2   sentence_tokenized  1057 non-null   object
 3   word_count          1057 non-null   int64 
 4   sentence_count      1057 non-null   int64 
 5   clean_words         1057 non-null   object
 6   clean_stemmed       1057 non-null   object
 7   clean_lemmed        1057 non-null   object
dtypes: int64(2), object(6)
memory usage: 66.2+ KB


In [16]:
# create the transform
vectorizer = CountVectorizer(stop_words = 'english', max_df = 0.75,ngram_range=(2, 4))
# tokenize and build vocab
bag_of_words = vectorizer.fit_transform(df.clean_lemmed)
# summarize
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:10]

[('year old', 269),
 ('locomotor disability', 153),
 ('data entry', 110),
 ('hearing impairment', 89),
 ('charitable trust', 85),
 ('ratna nidhi', 81),
 ('nidhi charitable', 78),
 ('ratna nidhi charitable', 78),
 ('nidhi charitable trust', 78),
 ('ratna nidhi charitable trust', 78)]

In [17]:
# 5379 documents represented as a 14,409 dimensional vector (14,409 words)
bag_of_words

<1057x115650 sparse matrix of type '<class 'numpy.int64'>'
	with 148392 stored elements in Compressed Sparse Row format>

In [18]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(bag_of_words)

LatentDirichletAllocation(n_components=5, random_state=42)

In [19]:
first_topic = LDA.components_[0]

In [20]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([94889, 94888, 94890, 60601, 18219, 85334, 85335, 85336, 70855,
       70854], dtype=int64)

In [21]:
for i in top_topic_words:
    print(vectorizer.get_feature_names()[i])

E:\Anaconda\envs\myenvnew\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


skilled ratna nidhi
skilled ratna
skilled ratna nidhi charitable
locomotor disability
charitable trust
ratna nidhi
ratna nidhi charitable
ratna nidhi charitable trust
nidhi charitable trust
nidhi charitable


In [22]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['skilled ratna nidhi', 'skilled ratna', 'skilled ratna nidhi charitable', 'locomotor disability', 'charitable trust', 'ratna nidhi', 'ratna nidhi charitable', 'ratna nidhi charitable trust', 'nidhi charitable trust', 'nidhi charitable']


Top 10 words for topic #1:
['best ability', 'spectrum disorder', 'autism spectrum disorder', 'looking opportunity', 'sign language', 'currently working', 'autism spectrum', 'hearing impairment', 'west bengal', 'year old']


Top 10 words for topic #2:
['course american', 'banking product', 'banking product like', 'insurance home', 'completed month training', 'completed month', 'type banking', 'soft skill', 'month training', 'year old']


Top 10 words for topic #3:
['month completed education', 'old candidate', 'year old candidate', 'polio age month', 'completed education', 'age month', 'affected polio age', 'affected polio', 'polio age', 'year old']


Top 10 words for topic #4:
['training trrain', 'andhra pradesh', 'complete

In [23]:
topic_values = LDA.transform(bag_of_words)
topic_values.shape

(1057, 5)

In [24]:
df['Topic'] = topic_values.argmax(axis=1)

In [25]:
df.head()

,bio,word_tokenized,sentence_tokenized,word_count,sentence_count,clean_words,clean_stemmed,clean_lemmed,Topic
0,ratul arora is from delhi. he has blindness. h...,"['ratul', 'arora', 'is', 'from', 'delhi', 'he'...","['ratul arora is from delhi.', 'he has blindne...",105,9,"['ratul', 'arora', 'delhi', 'blindness', 'hold...","['ratul', 'arora', 'delhi', 'blind', 'hold', '...","['ratul', 'arora', 'delhi', 'blindness', 'hold...",3
1,"aman verma is from nalasopara, maharashtra. he...","['aman', 'verma', 'is', 'from', 'nalasopara', ...","['aman verma is from nalasopara, maharashtra.'...",100,7,"['aman', 'verma', 'nalasopara', 'maharashtra',...","['aman', 'verma', 'nalasopara', 'maharashtra',...","['aman', 'verma', 'nalasopara', 'maharashtra',...",4
2,"sampa gupta is from mumbai, maharashtra. she h...","['sampa', 'gupta', 'is', 'from', 'mumbai', 'ma...","['sampa gupta is from mumbai, maharashtra.', '...",84,6,"['sampa', 'gupta', 'mumbai', 'maharashtra', 'b...","['sampa', 'gupta', 'mumbai', 'maharashtra', 'b...","['sampa', 'gupta', 'mumbai', 'maharashtra', 'b...",2
3,monu varma is a 29-year-old visually impaired ...,"['monu', 'varma', 'is', 'a', '29', 'year', 'ol...",['monu varma is a 29-year-old visually impaire...,89,5,"['monu', 'varma', '29', 'year', 'old', 'visual...","['monu', 'varma', '29', 'year', 'old', 'visual...","['monu', 'varma', '29', 'year', 'old', 'visual...",1
4,jayant singh raghav believes that the position...,"['jayant', 'singh', 'raghav', 'believes', 'tha...",['jayant singh raghav believes that the positi...,214,12,"['jayant', 'singh', 'raghav', 'believes', 'pos...","['jayant', 'singh', 'raghav', 'believ', 'posit...","['jayant', 'singh', 'raghav', 'belief', 'posit...",4
